In [1]:
import sys
sys.path.append("../src/")
from loss_function import loss_function
from loss_function import cs
from torch.autograd import Variable
from evaluate import Evaluation
from lstm import LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time

In [2]:
import data_reader as dr

In [3]:
corpus_path = "../data/text_tokenized.txt.gz"
corpus = dr.read_corpus(corpus_path)

In [4]:
embedding_path = "../data/vectors_pruned.200.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

In [5]:
ids_corpus = dr.map_corpus(corpus, word_to_indx, kernel_width = 1)

In [6]:
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)

In [7]:
train_ex = dr.create_train_set(ids_corpus, train)

In [8]:
def normalize(x, dim):
    l2 = torch.norm(x, 2, dim).expand_as(x)
    return x / l2.clamp(min = 1e-8)

In [11]:
import torch.utils.data

def train_model(train_data, dev_data, model):
    model.cuda()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0003, weight_decay = 1e-5)
    model.train()

    lasttime = time.time()
    for epoch in range(1, 11):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, model, optimizer)
        #return loss
        print('Train loss: {:.6f}'.format(loss))
        torch.save(model, "model{}".format(epoch))
        
        (MAP, MRR, P1, P5) = run_epoch(dev_data, False, model, optimizer)
        print('Val MAP: {:.6f}, MRR: {:.6f}, P1: {:.6f}, P5: {:.6f}'.format(MAP, MRR, P1, P5))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, model, optimizer):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=40,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []

    if is_training:
        model.train()
    else:
        model.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        pid_title_pad = torch.unsqueeze(Variable(batch['pid_title_pad']), 1)
        pid_body_pad = torch.unsqueeze(Variable(batch['pid_body_pad']), 1)
        rest_title_pad = Variable(batch['rest_title_pad'])
        rest_body_pad = Variable(batch['rest_body_pad'])
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        pid_title_pad, pid_body_pad = pid_title_pad.cuda(), pid_body_pad.cuda()
        rest_title_pad, rest_body_pad = rest_title_pad.cuda(), rest_body_pad.cuda()
        
        if is_training:
            optimizer.zero_grad()
        
        pt = model(pid_title)
        pb = model(pid_body)
        rt = model(rest_title)
        rb = model(rest_body)
        
        pt = normalize(pt, 2)
        pb = normalize(pb, 2)
        rt = normalize(rt, 2)
        rb = normalize(rb, 2)
        
        # we need to take the mean pooling taking into account the padding
        # tensors are of dim batch_size x samples x output_size x (len - kernel + 1)
        # pad tensors are of dim batch_size x samples x (len - kernel + 1)
        
        pid_title_pad_ex = torch.unsqueeze(pid_title_pad, 2).expand_as(pt)
        pid_body_pad_ex = torch.unsqueeze(pid_body_pad, 2).expand_as(pb)
        rest_title_pad_ex = torch.unsqueeze(rest_title_pad, 2).expand_as(rt)
        rest_body_pad_ex = torch.unsqueeze(rest_body_pad, 2).expand_as(rb)
        
        pt = torch.squeeze(torch.sum(pt * pid_title_pad_ex, dim = 3), dim = 3)
        pb = torch.squeeze(torch.sum(pb * pid_body_pad_ex, dim = 3), dim = 3)
        rt = torch.squeeze(torch.sum(rt * rest_title_pad_ex, dim = 3), dim = 3)
        rb = torch.squeeze(torch.sum(rb * rest_body_pad_ex, dim = 3), dim = 3)

        # tensors are of dim batch_size x samples x output_size
        # need to scale down because not all uniformly padded
        
        ptp_norm = torch.sum(pid_title_pad, dim = 2).clamp(min = 1).expand_as(pt)
        pbp_norm = torch.sum(pid_body_pad, dim = 2).clamp(min = 1).expand_as(pb)
        rtp_norm = torch.sum(rest_title_pad, dim = 2).clamp(min = 1).expand_as(rt)
        rbp_norm = torch.sum(rest_body_pad, dim = 2).clamp(min = 1).expand_as(rb)
        
        pt = pt / ptp_norm
        pb = pb / pbp_norm
        rt = rt / rtp_norm
        rb = rb / rbp_norm
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        
        if is_training:
            dropout = nn.Dropout(p = 0.2)
            # we don't need to re-scale these on eval because its just cos sim
            pid_tensor = dropout(pid_tensor)
            rest_tensor = dropout(rest_tensor)
        
        if is_training:
            loss = loss_function(pid_tensor, rest_tensor, margin = 1.0)
            loss.backward()
            losses.append(loss.cpu().data[0])
            optimizer.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2).squeeze(2)
            similarity = similarity.data.cpu().numpy()
            #return similarity
            labels = batch['labels'].numpy()
            l = dr.convert(similarity, labels)
            losses.extend(l)

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        return avg_loss
    else:
        e = Evaluation(losses)
        MAP = e.MAP()*100
        MRR = e.MRR()*100
        P1 = e.Precision(1)*100
        P5 = e.Precision(5)*100
        return (MAP, MRR, P1, P5)

In [12]:
model = LSTM(310, embedding_tensor, 0.2)

In [13]:
val_path = "../data/dev.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

In [14]:
z = train_model(train_ex, val_ex, model)

-------------
Epoch 1:

Train loss: 1.021194
Val MAP: 50.556219, MRR: 62.686402, P1: 49.206349, P5: 41.058201
This epoch took: 100.651429
-------------
Epoch 2:

Train loss: 0.951551
Val MAP: 54.677259, MRR: 66.116990, P1: 52.380952, P5: 43.597884
This epoch took: 100.108730
-------------
Epoch 3:

Train loss: 0.907675
Val MAP: 54.822690, MRR: 67.573342, P1: 54.497354, P5: 43.068783
This epoch took: 98.035828
-------------
Epoch 4:

Train loss: 0.889985
Val MAP: 55.364708, MRR: 68.884833, P1: 57.142857, P5: 44.656085
This epoch took: 98.060693
-------------
Epoch 5:

Train loss: 0.882005
Val MAP: 55.777860, MRR: 68.695958, P1: 55.555556, P5: 44.550265
This epoch took: 97.912830
-------------
Epoch 6:

Train loss: 0.873461
Val MAP: 55.848841, MRR: 69.110827, P1: 57.142857, P5: 44.338624
This epoch took: 97.958396
-------------
Epoch 7:

Train loss: 0.868064
Val MAP: 57.159214, MRR: 70.298158, P1: 58.730159, P5: 46.137566
This epoch took: 97.929060
-------------
Epoch 8:

Train loss: 0.8

In [21]:
model = torch.load("model4")
real_val_path = "../data/test.txt"
real_val = dr.read_annotations(real_val_path, K_neg = -1, prune_pos_cnt = -1)
real_val_ex = dr.create_dev_set(ids_corpus, real_val)
model = model.cuda()
z = run_epoch(real_val_ex, False, model, None)

In [22]:
print(z)

(57.49701865047421, 71.24889516524142, 57.52688172043011, 43.01075268817206)


In [23]:
tot = 0
for s in filter(lambda p: p.requires_grad, model.parameters()):
    tot += np.prod(s.size())
print(tot)

443920
